In [2]:
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2


In [74]:
# (1) Derive Probabilties for transition

from src.modules.probabilities import Probabilities

np.random.seed(14)
prob = Probabilities()

p = prob.getProbabilities(1*con.tau*60)

#sum(p["p"]) # == 1 !


In [35]:
from src.modules import constants as con
# Construct ex_info data frame for all ts
df_p = pd.DataFrame()

for t in np.arange(0, 1440, con.tau*60):
    df_p = df_p.append(prob.getProbabilities(t))
    
df_p.tail()

,Trip,Length,Price,p,t
77,y,38,20,1.12251e-08,1425.0
78,y,39,10,0,1425.0
79,y,39,20,0,1425.0
80,n,0,10,0.997235,1425.0
81,n,0,20,0.00207953,1425.0


In [2]:
# (2) Derive state space

import modules.generator as g
import time
start = time.process_time()
# your code here    
sts = g.constructStates()
print(time.process_time() - start)
sts.describe()

18887it [00:00, 188850.82it/s]

4782996


4782996it [00:33, 144839.88it/s]


70.49084140000001


,t,B_L,V_TA,D,P_B,P_S,V_N
count,4.782996e+06,4.782996e+06,4.782996e+06,4.782996e+06,4.782996e+06,4.782996e+06,4782996.0
mean,4.500869e+00,3.000237e+01,1.499763e+00,1.999684e+01,2.500000e-01,2.500000e-01,0.0
std,2.872887e+00,1.169111e+01,1.118105e+00,1.183390e+01,1.707825e-01,1.707825e-01,0.0
min,0.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,2.000000e+00,2.000000e+01,0.000000e+00,1.000000e+01,1.000000e-01,1.000000e-01,0.0
50%,5.000000e+00,3.000000e+01,1.000000e+00,2.000000e+01,2.500000e-01,2.500000e-01,0.0
75%,7.000000e+00,4.000000e+01,2.000000e+00,3.000000e+01,4.000000e-01,4.000000e-01,0.0
max,1.000000e+01,5.000000e+01,3.000000e+00,4.000000e+01,5.000000e-01,5.000000e-01,0.0


In [17]:
sts.head()
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 2) & (sts["B_L"] >= 40) & (sts["D"] == 0)]) ##120
len(sts[(sts["V_TA"] == 0) & (sts["t"] == 10) & (sts["B_L"] >= 40)]) ##120

756

In [31]:
# (3) Derive decision space
from tqdm import tqdm
dec = {}

for s in tqdm(sts["obj"].head(1)):
    dec[s.getKey()] = g.constructDecisions(s)

#print(sts.iloc[0])
#print(dec)
print(pd.DataFrame([x.__str__() for x in dec[sts.loc[0, "obj"].getKey()]]))

100%|██████████| 1/1 [00:00<00:00, 35.29it/s]

6
4
4
4
4
4
2
2
2
2
2
2
2
2
2
2
                                  0
0  Decision [(0.000000,0.000000,0)]
1  Decision [(0.500000,0.000000,0)]
2  Decision [(1.000000,0.000000,0)]
3  Decision [(1.500000,0.000000,0)]
4  Decision [(2.000000,0.000000,0)]
5  Decision [(2.500000,0.000000,0)]


In [84]:
# Perform transitions

from modules.transition import Transition
from src.modules.probabilities import Probabilities

np.random.seed(14)

prob = Probabilities()
ex_inf = prob.getProbabilities(300).loc[0,:]
t = Transition(None, sts.loc[0, "obj"], dec[sts.loc[0,"obj"].getKey()][-1], ex_inf["p"], ex_inf[0:3])


print(sts.loc[0, "obj"])
print(dec[sts.loc[0,"obj"].getKey()][-1])
print(ex_inf)
print()
print(t.get_s_d().__str__())

0.0
State t=0[(10.0,0.0, 0.0,0.0,0.0) - False - (0.000000, 0.000000)]
Decision [(2.500000,0.000000,0)]
Trip                y
Length              0
Price              10
p         7.68354e-05
Name: 0, dtype: object

State t=1[(12.25,0.0, 0,10,10) - False - (0.000000, 0.000000)]


In [78]:
from src.modules.app import App

a = App()
a.valueIteration()

16967it [00:00, 169655.20it/s]

4782996


4782996it [00:25, 189523.49it/s]


6
4
4
4
4
4
2
2
2
2
2
2
2
2
2
2


TypeError: unhashable type: 'State'